In [7]:
import pandas as pd
import numpy as np
import datetime
from indicators import *
import os

In [3]:
from candle_creation_new import *

In [8]:
from all_stochastic_functions import *

In [11]:
from all_fractal_functions import *

In [119]:
files = get_all_file_paths("C:\\Users\\TEMP002\\Downloads\\Complete Dataset\\CSV\\NIFTY BANK","SPOT",2016,2022)
total_data = get_dataset(files)

In [120]:
total_data.head(5)

,Time,Open,High,Low,Close
time_stamp,,,,,
2016-10-20 09:15:00,09:15:00,19491.3,19526.3,19479.3,19502.2
2016-10-20 09:16:00,09:16:00,19501.3,19521.7,19499.3,19521.1
2016-10-20 09:17:00,09:17:00,19525.2,19532.1,19514.0,19521.9
2016-10-20 09:18:00,09:18:00,19521.6,19538.3,19519.9,19532.1
2016-10-20 09:19:00,09:19:00,19531.6,19542.9,19530.2,19536.4


In [121]:
def fractals(data):
    
    data['prev_2_high']=data['High'].rolling(3).max()
    data['prev_2_low'] = data['Low'].rolling(3).min()
    data['next_high'] = data['High'].shift(-1)
    data['next_low'] = data['Low'].shift(-1)
    data['last_high'] = data['High'].shift(-2)
    data['last_low'] = data['Low'].shift(-2)

    data['HF_appeared'] = data.apply(lambda x: high_fractal_appeared(x['High'],x['prev_2_high'],x['next_high']),axis=1)
    data['LF_appeared'] = data.apply(lambda x: low_fractal_appeared(x['Low'],x['prev_2_low'],x['next_low']),axis=1)

    data['is_high'] = data.apply(lambda x: high_fractal_broken(x['HF_appeared'],x['High'],x['last_high']),axis=1)
    data['is_low'] = data.apply(lambda x : low_fractal_broken(x['LF_appeared'],x['Low'],x['last_low']),axis=1)

    data['fractal_time'] = data.index
    data['fractal_time'] = data['fractal_time'].shift(-2)
    data.loc[(data['is_high']==0) & (data['is_low']==0),'fractal_time'] = np.NaN
    
    data.drop(columns=['prev_2_high','prev_2_low','next_high','next_low','last_high','last_low'],axis=1,inplace=True)
    return data

In [122]:
def stochastic(data, k_length=14, d_length=3, ewm=9):
    """
    Takes K_length , D_length, EWM span as inputs
    """
    data['14-H'] = data['High'].rolling(k_length).max()
    data['14-L'] = data['Low'].rolling(k_length).min()
    data['%_K'] = (data['Close']-data['14-L'])/(data['14-H']-data['14-L'])*100
    data['%_D'] = data['%_K'].rolling(d_length).mean()
    data['%_K_EMA'] = data['%_K'].ewm(span=ewm,min_periods=0,adjust=False,ignore_na=False).mean()
    data.drop(columns=['14-H','14-L'],inplace=True)
    return data

In [123]:
def ema_bin_wise_fractal_avg(data,bin_col,bin_val,green_fractal_col_name = 'fractal_avg_green',red_fractal_col_name='fractal_avg_red'):
    
    # data['level'] = np.where(data['EMA_bins']==bin_val,1,0)   ##2
    data['level'] = np.where(data[bin_col]==bin_val,1,0)
    # data.loc[data['EMA_bins'].isna(),'level'] = np.NaN
    data.loc[data[bin_col].isna(),'level'] = np.NaN
    
    data['level_change'] = data['level'] - data['level'].shift()
    
    first_row = data[data[bin_col]==bin_val]
    if not first_row.empty:
        time_stamp = data[data[bin_col]==bin_val].iloc[0].name
        data.at[time_stamp,'level_change'] = 1

    thresh_cross_indices = data[data['level_change']==1].index 
    cross_limit = 50
    
    data = generate_green_fractal_average(data,cross_limit,thresh_cross_indices,green_fractal_col_name)
    data = generate_red_fractal_average(data,cross_limit,thresh_cross_indices,red_fractal_col_name)

    data = data.drop(columns=['level','level_change'])  ##fropping level and level_change columns
    
    return data

In [124]:
def stocastic_functions(data,bin_length = 20):
    data=interval_diff(data,'%_K_EMA','%K_Int_Diff')
    data=classification_1(data,'%K_Int_Diff','EMA classification_1')
    data=classification_2(data,'%K_Int_Diff','EMA classification_2') 

    data=binning_EMA(data,'%_K_EMA',bin_length,'EMA_bins_20')                                   #creating bins on %K_EMA by bounds which are multiple of input
    data = classification_2_bin_wise(data,'EMA classification_2','EMA_bins_20','EMA classification_7_20bin')      ##if current row Class-2 = prev row Class 2 and bins are matching altering current Class-2 to prev Class-2

    data = ema_bin_wise_fractal_avg(data,'EMA_bins_20','81-100',green_fractal_col_name = 'fractal_avg_green_20bin>80',red_fractal_col_name='fractal_avg_red_20bin>80')
    data = ema_bin_wise_fractal_avg(data,'EMA_bins_20','<20',green_fractal_col_name = 'fractal_avg_green_20bin<20',red_fractal_col_name='fractal_avg_red_20bin<20')

    return data

In [125]:
def fractal_now(data, trailing_avg_length = 5, bin_threshold = 0.5, trailing_length = 10, trailing_stride = 2, trailing_continuos = 5):

    data = get_n_trailing_averages_new(data,trailing_avg_length)

    data['fractal_val_green'] = data.apply(lambda row : get_fractal_val_green(row.is_high,row.High,row.Low),axis=1)
    data['fractal_val_red'] = data.apply(lambda row : get_fractal_val_red(row.is_low,row.High,row.Low),axis=1)

    data['abs_diff'] = data.apply(lambda row: compute_abs_diff(row.is_low,row.is_high,row.prev_n_op_red_avg,row.prev_n_op_green_avg,row.fractal_val_green,row.fractal_val_red),axis=1)
    data['%_growth'] = data.apply(lambda row: compute_percent_growth(row.is_low,row.is_high,row.prev_n_op_red_avg,row.prev_n_op_green_avg,row.fractal_val_green,row.fractal_val_red),axis=1)

    ## modifying the values for 1,1 fractals
    neutral_indices = data[(data['is_high']==1) & (data['is_low']==1)].index

    green_start = data['prev_n_op_red_avg'].first_valid_index()
    red_start = data['prev_n_op_green_avg'].first_valid_index()

    if green_start<red_start:
        start_index = green_start
    else:
        start_index = red_start


    neutral_index_is_high = pd.Series(dtype="float64",index=data.index)

    for index in neutral_indices:
        prev_total_data = data.loc[start_index:index]
        prev_normal_data = prev_total_data[~((prev_total_data['is_high']==1) & (prev_total_data['is_low']==1))]

        if not prev_normal_data.empty:
            prev_last_normal_rec = prev_normal_data.iloc[-1]

            if prev_last_normal_rec.is_high:
                data.at[index,'abs_diff'] = data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg']
                data.at[index,'%_growth'] = (data.loc[index]['fractal_val_green']/data.loc[index]['prev_n_op_red_avg'] - 1)*100
                neutral_index_is_high[index] = 1
            else:
                data.at[index,'abs_diff'] = data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg']
                data.at[index,'%_growth'] = (data.loc[index]['fractal_val_red']/data.loc[index]['prev_n_op_green_avg'] - 1)*100
                neutral_index_is_high[index] = 0
        
        
        else:
            if (data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg']) < 0:
                data.at[index,'abs_diff'] = (data.loc[index]['fractal_val_green'] - data.loc[index]['prev_n_op_red_avg'])
                data.at[index,'%_growth'] = (data.loc[index]['fractal_val_green']/data.loc[index]['prev_n_op_red_avg'] - 1)*100
                neutral_index_is_high[index] = 1

            elif (data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg']) > 0:
                data.at[index,'abs_diff'] = (data.loc[index]['fractal_val_red'] - data.loc[index]['prev_n_op_green_avg'])
                data.at[index,'%_growth'] = (data.loc[index]['fractal_val_red']/data.loc[index]['prev_n_op_green_avg'] - 1)*100
                neutral_index_is_high[index] = 0
            else:
                data.at[index,'abs_diff'] = np.NaN

    data = data.assign(neutral_index_is_high = neutral_index_is_high)

    data,label_map,op_map = generate_levels(data,'abs_diff')
    data = bin_abs_diff(data,label_map,bin_threshold)
    data,new_map = step_5_new(data,label_map)
    data = step_6(data,new_map)
    data = calc_step_7(data,new_map)
    return data

In [126]:
data = get_resampled_data(total_data,'25T')

In [127]:
data = stochastic(data,150,3,9)
data = fractals(data)
data = stocastic_functions(data)


2017-01-06T09:15:00.000000000
2017-03-14T12:35:00.000000000
2017-03-28T13:00:00.000000000
2017-04-25T09:15:00.000000000
2017-05-26T09:15:00.000000000
2017-06-19T14:15:00.000000000
2017-07-13T10:30:00.000000000
2017-08-28T13:00:00.000000000
2017-09-01T10:05:00.000000000
2017-10-10T09:15:00.000000000
2017-10-25T14:40:00.000000000
2017-11-17T11:20:00.000000000
2017-12-15T12:35:00.000000000
2018-01-08T10:05:00.000000000
2018-04-04T09:15:00.000000000
2018-07-09T11:45:00.000000000
2018-07-26T10:05:00.000000000
2018-08-27T15:05:00.000000000
2018-11-01T15:05:00.000000000
2019-01-01T15:05:00.000000000
2019-02-06T12:10:00.000000000
2019-03-05T11:20:00.000000000
2019-05-20T10:55:00.000000000
2019-06-26T11:45:00.000000000
2019-09-12T12:35:00.000000000
2019-09-20T13:50:00.000000000
2019-10-17T14:15:00.000000000
2019-12-13T13:00:00.000000000
2020-01-13T12:10:00.000000000
2020-02-06T12:10:00.000000000
2020-05-29T11:20:00.000000000
2020-06-22T09:40:00.000000000
2020-08-19T10:30:00.000000000
2020-10-01

In [128]:
filter_1 = data['is_high'] == 1
filter_2 = data['is_low'] == 1
fractal_data = data[filter_1 | filter_2]

fractal_data = fractal_data[['Open','High','Low','Close','is_high','is_low']]
#fractal_data = get_n_trailing_averages_new(fractal_data,win_len=5)


In [129]:
fractal_data = fractal_now(fractal_data,trailing_avg_length=5)

c:\Users\TEMP002\Desktop\INDICATORS\FUNCTIONS\all_fractal_functions.py:320: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_A_firsts['shift'] = only_A_firsts[step_7_map[new_map['A']]].shift(step)   #Shifting trailing average
c:\Users\TEMP002\Desktop\INDICATORS\FUNCTIONS\all_fractal_functions.py:321: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_B_firsts['shift'] = only_B_firsts[step_7_map[new_map['B']]].shift(step)
c:\Users\TEMP002\Desktop\INDICATORS\FUNCTIONS\all_fractal_functions.py:323: SettingWi

In [130]:
fractal_data = fractal_data.drop(columns=['Open','High','Low','Close','is_high','is_low'])

final = data.join(fractal_data)


In [131]:
##Adjusting level column so that we have a value at non-fractal candles

start_index = final.loc[final['level_change']==1].iloc[0].name

for index,row  in final[start_index:].iterrows():
    if row.level_change == 1:
        current_level = row['level']
    
    if row.level != row.level:
        final.at[index,'level'] = current_level

In [132]:
final.to_excel('./25TF_150K_fractal_5_bank_nifty.xlsx')

In [118]:
fractal_data[['is_high','is_low','neutral_index_is_high','abs_diff','level','level_change']][50:100]

,is_high,is_low,neutral_index_is_high,abs_diff,level,level_change
time_stamp,,,,,,
2016-11-07 09:15:00,1,0,NaN,59.69,A,0.0
2016-11-07 10:30:00,1,0,NaN,59.64,A,0.0
2016-11-07 11:45:00,0,1,NaN,-10.43,A,0.0
2016-11-07 13:00:00,1,0,NaN,69.47,A,0.0
2016-11-07 15:05:00,0,1,NaN,-29.38,A,0.0
2016-11-08 09:15:00,1,0,NaN,80.15,A,0.0
2016-11-08 10:30:00,0,1,NaN,-29.69,A,0.0
2016-11-08 11:20:00,1,0,NaN,49.67,A,0.0
2016-11-08 13:25:00,0,1,NaN,-46.96,A,0.0
